In [1]:
import pandas as pd
import fitz
import re

In [2]:
doc=fitz.open('../papers/Designing of A Based Route Recommendation Service for Multimodal Transportation System in Smart Cities.pdf')

In [55]:
def correct_name(match):
    return text[match.start(): match.end()].replace('.', '').strip()
found_references=False
pattern = r'(?<=\(\d\d\d\d\)).*?(?=\.|\n)'
references=[]
for i, page in enumerate(doc):
    text = page.get_text().replace(' \n', ' ').replace('-\n', '')
    if ("References\n" in text) or found_references:
        if ("References\n" in text):
            ini = text.find("References\n")
            text = text[ini+11:]
        found_references=True
        citations = re.finditer(pattern, text)
        citations = [correct_name(match) for match in citations if len(correct_name(match))!=0]
        references+=citations

In [57]:
references

['An algorithmic framework for computing shortest routes in urban multimodal networks with different criteria',
 'The project of intellectual multimodal transport system',
 'Intelligent traffic congestion classification system using artificial neural network',
 'Transfer graph approach for multimodal transport problems',
 'Fast routing in very large public transportation networks using transfer patterns',
 'Multiple criteria decision analysis: an integrated approach',
 'Managing smart-city transportation planning of ‘park-and-ride’ system: case of moscow metropolitan',
 'Accelerating multi-modal route planning by access-nodes',
 'A multimodal transport network model and efficient algorithms for building advanced traveler information systems',
 'User-constrained multimodal route planning',
 'Algorithm\xa0 97: shortest path',
 'Resilience in intelligent transportation systems (its)',
 'A distributed approach for shortest path algorithm in dynamic multimodal transportation networks',
 'Un